<img src="../static/logopython.png" alt="Logo Python" style="width: 300px; display: inline"/>
<img src="../static/deimoslogo.png" alt="Logo Deimos" style="width: 300px; display: inline"/>

# Clase 5: Ejercicios prácticos

En esta clase vamos a afianzar los conocimientos de Python que acabamos de adquirir haciendo algunos ejercicios sobre las herramientas del lenguaje Python para trabajar con metaprogramación


## Ejercicio 1

Escribe un decorador para una función que informe del tiempo de ejecución de la misma

__Pista__: Investiga si algún método del módulo [time](https://docs.python.org/3.0/library/time.html) te puede resultar útil

In [ ]:
# Tu codigo aqui.
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [ ]:
# Codigo de pruebas. Deberias decorar ese metodo para que, al llamarlo, imprimiera el tiempo de ejecucion
@timethis
def countdown(n):
    """
    Una simple cuenta atras
    """
    while n > 0:
        n -= 1
        
countdown(10000000)

# Acuérdate: ¿qué debería hacer en mi decorador para que aquí me informara correctamente de lo que hace mi método?
help(countdown)

## Ejercicio 2

Escribe un decorador de función que añada un mensaje de log cada vez que se llame a la función decorada. Las condiciones son:

* Debes usar la clase [logging](https://docs.python.org/3/howto/logging.html) para mostrar el mensaje.
* El decorador debe aceptar dos argumentos: el nivel de log (dentro de los niveles aceptados por logging) y un mensaje de texto
* Al imprimir por pantalla el mensaje, debe ir precedido por el nombre de la función decorada

__Pista__: Revisa los métodos de la clase [Logger](https://docs.python.org/3/howto/logging.html#loggers), para ver cuál te deja pasarle un nivel como argumento

In [72]:
# Tu codigo aqui
import logging
from functools import wraps

logging.basicConfig(level=logging.INFO)

def logged(level, message):
    '''
    Añadiendo logging a una funcion. level es el
    nivel de logging, y message es el mensaje de log
    '''
    def decorate(func):
        log = logging.getLogger(func.__name__)
        logmsg = message

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [74]:
# Codigo de pruebas
@logged(logging.DEBUG, "Esto es un mensaje de debug")
def add(x, y):
    return x + y

@logged(logging.CRITICAL, "Esto es un mensaje critico")
def spam():
    print('Spam!')
    
# Si ponemos el nivel de log a INFO, el mensaje de DEBUG no debería salir. Y en el interprete no sale
# pero aqui en el notebook si...
logging.basicConfig(level=logging.INFO)
    
spam()
add(3, 4)




Spam!


7

## Ejercicio 3

Modifica el código de la siguiente función para que acepte un parámetro opcional llamado *also_privates*. Dicho parámetro se evaluará a *True* o *False* dentro de la función, y servirá para determinar si se muestran o no también los métodos *privados* del objeto que se le pase (recuerda que consideramos *privados* aquellos métodos cuyo nombre comienza por \__)

In [37]:
def info(object, also_privates=False): 
    """
        Imprime todos los metodos del objeto y sus docstrings.
        Acepta modulos, clases, listas, diccionarios y cadenas
    """
    methodList = [method for method in dir(object) if callable(getattr(object, method))]
    if not also_privates:
        methodList = filter(lambda s: not s.startswith('__'), methodList)
        
    print("\n".join(["{} {}".format
                      (method,
                       str(getattr(object, method).__doc__))
                     for method in methodList]))



In [62]:
class Punto:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def foo():
        "Metodo absurdo"
        print("bar")

info(Punto)
        
# Descomenta esto cuando termines de modificar la funcion
#info(Punto, also_privates=True)

foo Metodo absurdo


## Ejercicio 4

Escribe una clase totalmente genérica. La clase debe cumplir estas condiciones:

* Su constructor debe aceptar cualquier número de parámetros con nombre, y debe crear por cada pareja clave=valor, un atributo de instancia con el nombre *clave* y el valor *valor*. 
* Al pasar una instancia de la clase como argumento de la función *print*, se debe mostrar una cadena que incluya el nombre de la clase y, a continuación, todos los atributos separados por comas

__Pista__: Recuerda la utilización de \**kwargs para pasar parámetros con nombre a una función 


In [48]:
# Tu codigo aqui
class ClaseGenerica:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

    def __str__(self):
        attrs = ["{}={}".format(k, v) for (k, v) in self.__dict__.items()]
        classname = self.__class__.__name__
        return "%s: %s" % (classname, ",".join(attrs))

In [50]:
# Codigo de pruebas
c = ClaseGenerica(foo='bar', x=3, y=4, z=5)

print(c.foo)
print(c.x)
print(c.y)
print(c.z)
print(c)

bar
3
4
5
ClaseGenerica: foo=bar,z=5,x=3,y=4


## Ejercicio 5

Considera la siguiente definición de una clase

```class Stock:
        def __init__(self, name, shares, price):
            self.name = name
            self.shares = shares
            self.price = price```
            
Con las técnicas de metaprogramación que hemos visto, implementa mecanismos para:

* Shares sea siempre un número de tipo float, y con valor entre 0 y 1
* Price sea un valor de tipo float mayor que 0
* No se pueda borrar ninguno de los campos


In [60]:
# Tu codigo aqui

# Descriptor basico
class Descriptor:
  # No nos hace falta get solo devolvería el valor
  def __init__(self, name=None):
    self.name = name
  def __set__(self, instance, value):
    instance.__dict__[self.name] = value

  # No dejamos borrar ningún campo
  def __delete__(self, instance):
    raise AttributeError("Can't delete")
    
# Clases para la comprobación de tipos
class Typed(Descriptor):
    ty = object
    def __set__(self, instance, value):
        if not isinstance(value, self.ty):
            raise TypeError('Expected {}'.format(self.ty))
        super().__set__(instance, value)
        
class Integer(Typed):
    ty = int
    
class Float(Typed):
    ty = float
    
    
# Clases para la comprobación de valores
class Positivo(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)
        
class Porcentaje(Descriptor):
    def __set__(self, instance, value):
        if value < 0.0 or value > 1.0:
            raise ValueError('Expected between 0.0 and 1.0')
        super().__set__(instance, value)
        
        
# Combinando
class PorcentajeFloat(Float, Porcentaje):
    pass

class EnteroPositivo(Integer, Positivo):
    pass


# Definimos nuestra clase
class Stock:
        def __init__(self, name, shares, price):
            self.name = name
            self.shares = shares
            self.price = price
            
        shares = PorcentajeFloat('shares')
        price = EnteroPositivo('price')
        name = Descriptor('name')

In [61]:
# Codigo de prueba

s = Stock("Microsoft", 0.78, 12)

# Operaciones que no se deben poder hacer
s.shares = 132.12
s.price = -12
del s.name

AttributeError: Can't delete

##### <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es"><img alt="Licencia Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Curso Python</span> por <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Jorge Arévalo</span> se distribuye bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es">Licencia Creative Commons Atribución 4.0 Internacional</a>.

---
_Las siguientes celdas contienen configuración del Notebook_

_Para visualizar y utlizar los enlaces a Twitter el notebook debe ejecutarse como [seguro](http://ipython.org/ipython-doc/dev/notebook/security.html)_

    File > Trusted Notebook

In [39]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_file = '../static/styles/style.css'
HTML(open(css_file, "r").read())